In [1]:
%matplotlib inline
%load_ext tensorboard
#!pip install keras-tuner
import pandas as pd
import numpy as np
import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
from tensorflow.keras.layers import Layer
for gpu in tf.config.list_physical_devices("GPU"):
    tf.config.experimental.set_memory_growth(gpu, True)
from matplotlib import pyplot as plt
import keras_tuner as kt
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.python.ops.numpy_ops import np_config

2024-07-01 13:52:59.977084: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-01 13:53:00.276097: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-01 13:53:01.506595: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype
#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 0]#index=0 --> 1_CobaltChloride_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

genotypeFile shape: (4390, 28220)
Phenotype_Multi shape: (4390, 20)


In [84]:
corrupted_X_onehot.shape

(4390, 28220, 3)

In [3]:
x_test.shape

(395, 28220, 3)

In [4]:
background_list = list([i for i in range(404)])[0:100]
if len(background_list)==0:
    background_list = x_train[np.random.choice(x_train.shape[0],100, replace=False)]
background = x_train[background_list]
background.shape

(100, 28220, 3)

In [5]:
import shap
from shap import initjs
initjs()
custom_objects = None
# Define custom objects if necessary
custom_objects = None

# Load the model with specified reduction strategy
model_mse = tf.keras.models.load_model('1_CobaltChloride_1.h5', custom_objects=custom_objects, compile=False)


The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
2024-07-01 13:55:56.859240: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11557 MB memory:  -> device: 0, name: Quadro RTX 5000, pci bus id: 0000:19:00.0, compute capability: 7.5
2024-07-01 13:55:56.860003: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13633 MB memory:  -> device: 1, name: Quadro RTX 5000, pci bus id: 0000:1a:00.0, compute capability: 7.5
2024-07-01 13:55:56.860549: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 13633 MB memory:  -> device: 2, name: Quadro RTX 5000, pci bus i

In [6]:
model_mse.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28220, 3)]        0         
                                                                 
 conv1d (Conv1D)             (None, 9405, 8)           176       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 4702, 8)          0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 1566, 8)           456       
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 783, 8)           0         
 1D)                                                             
                                                                 
 flatten (Flatten)           (None, 6264)              0     

In [7]:
explainer = shap.DeepExplainer(model_mse,x_test)
base_value = explainer.expected_value

keras is no longer supported, please use tf.keras instead.
Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode. See PR #1483 for discussion.
2024-07-01 13:56:08.288525: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8700
2024-07-01 13:56:08.767034: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [8]:
base_values = base_value[0]

In [9]:
shap_values = explainer.shap_values(x_test,check_additivity=False)

In [10]:
mean_shap_values = np.mean(shap_values, axis=0)

In [11]:
len(mean_shap_values[0])

28220

In [33]:
abs_shap_values = np.abs(shap_values[0])
feature_importance = np.mean(abs_shap_values, axis=0)
feature_importance

array([[0.00000000e+00, 1.86048785e-04, 1.51459145e-04],
       [0.00000000e+00, 5.91382214e-04, 4.08680743e-05],
       [0.00000000e+00, 2.77450941e-04, 6.46576182e-04],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [13]:
feature_importance.shape

(28220, 3)

In [34]:
feature_impportance_sum = feature_importance.sum(axis=1)
feature_impportance_sum

array([0.00033751, 0.00063225, 0.00092403, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
X.shape

In [38]:
feature_importance_df = pd.DataFrame({
    'Feature': X.columns.tolist(),  # Convert X.columns to a list
    'Importance': feature_impportance_sum.tolist()
})

#feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
feature_importance_df['Feature'].tolist()

['33070_chrI_33070_A_T',
 '33147_chrI_33147_G_T',
 '33152_chrI_33152_T_C',
 '33200_chrI_33200_C_T',
 '33293_chrI_33293_A_T',
 '33328_chrI_33328_C_A',
 '33348_chrI_33348_G_C',
 '33403_chrI_33403_C_T',
 '33502_chrI_33502_A_G',
 '33548_chrI_33548_A_C',
 '33612_chrI_33612_A_G',
 '33892_chrI_33892_C_G',
 '33996_chrI_33996_A_G',
 '34170_chrI_34170_T_A',
 '34308_chrI_34308_C_T',
 '34515_chrI_34515_T_C',
 '34554_chrI_34554_A_G',
 '34943_chrI_34943_C_G',
 '34991_chrI_34991_A_G',
 '35059_chrI_35059_G_A',
 '35070_chrI_35070_C_T',
 '35129_chrI_35129_G_A',
 '35466_chrI_35466_G_A',
 '35508_chrI_35508_A_G',
 '35514_chrI_35514_T_C',
 '35565_chrI_35565_A_G',
 '35568_chrI_35568_A_G',
 '35631_chrI_35631_G_C',
 '35668_chrI_35668_G_A',
 '35736_chrI_35736_G_A',
 '35751_chrI_35751_T_C',
 '35818_chrI_35818_A_G',
 '35970_chrI_35970_T_C',
 '36005_chrI_36005_A_G',
 '36048_chrI_36048_C_G',
 '36123_chrI_36123_C_T',
 '36160_chrI_36160_G_A',
 '36313_chrI_36313_A_G',
 '36524_chrI_36524_C_T',
 '36574_chrI_36574_A_G',


In [28]:
num = feature_importance_df[feature_importance_df['Importance']  <= 1e-4]
less_important_features = num.Feature.tolist()
len(less_important_features)

576

In [36]:
num = feature_importance_df[feature_importance_df['Importance']  >= 1e-4]
important_features = num.Feature.tolist()
important_features

['33070_chrI_33070_A_T',
 '33147_chrI_33147_G_T',
 '33152_chrI_33152_T_C',
 '33200_chrI_33200_C_T',
 '33293_chrI_33293_A_T',
 '33328_chrI_33328_C_A',
 '33348_chrI_33348_G_C',
 '33403_chrI_33403_C_T',
 '33502_chrI_33502_A_G',
 '33548_chrI_33548_A_C',
 '33612_chrI_33612_A_G',
 '33892_chrI_33892_C_G',
 '33996_chrI_33996_A_G',
 '34170_chrI_34170_T_A',
 '34308_chrI_34308_C_T',
 '34515_chrI_34515_T_C',
 '34554_chrI_34554_A_G',
 '34943_chrI_34943_C_G',
 '34991_chrI_34991_A_G',
 '35059_chrI_35059_G_A',
 '35070_chrI_35070_C_T',
 '35129_chrI_35129_G_A',
 '35466_chrI_35466_G_A',
 '35508_chrI_35508_A_G',
 '35514_chrI_35514_T_C',
 '35565_chrI_35565_A_G',
 '35568_chrI_35568_A_G',
 '35631_chrI_35631_G_C',
 '35668_chrI_35668_G_A',
 '35736_chrI_35736_G_A',
 '35751_chrI_35751_T_C',
 '35818_chrI_35818_A_G',
 '35970_chrI_35970_T_C',
 '36005_chrI_36005_A_G',
 '36048_chrI_36048_C_G',
 '36123_chrI_36123_C_T',
 '36160_chrI_36160_G_A',
 '36313_chrI_36313_A_G',
 '36524_chrI_36524_C_T',
 '36574_chrI_36574_A_G',


In [42]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_shap = find_indexes(feature_importance_df['Feature'].tolist(), important_features)
print(len(indexes_shap))

27644


In [22]:
feature_importance_dict = dict(zip(feature_importance_df['Feature'], feature_importance_df['Importance']))
feature_importance_dict

{'5426582_chrIX_34009_T_C': 0.003270955726466012,
 '7857001_chrXII_611973_G_A': 0.003214025773075586,
 '10178520_chrXV_146551_T_A': 0.002975064121581661,
 '7190519_chrXI_612307_C_T': 0.002929086761904256,
 '10203640_chrXV_171671_A_G': 0.0029234676119663564,
 '10197796_chrXV_165827_C_T': 0.002907926153761476,
 '10324852_chrXV_292883_A_G': 0.0027239680483265296,
 '10197327_chrXV_165358_T_C': 0.0026523655883500374,
 '11388224_chrXVI_264964_C_T': 0.002607143375745538,
 '6374769_chrX_542308_A_G': 0.002594160797852504,
 '1640320_chrIV_280298_T_A': 0.0025886586843372715,
 '10199078_chrXV_167109_T_C': 0.002579993662266709,
 '7858876_chrXII_613848_C_G': 0.0025445512117953572,
 '7268596_chrXII_23568_T_C': 0.0025410674716113135,
 '10196677_chrXV_164708_A_G': 0.002533311683657326,
 '7191087_chrXI_612875_T_C': 0.0025142510101938535,
 '8404026_chrXIII_80821_A_C': 0.0024958240174518197,
 '6367891_chrX_535430_T_C': 0.002472049684600486,
 '6689046_chrXI_110834_T_C': 0.0024479768745147307,
 '5033772_chr

In [ ]:
sorted_features = sorted(feature_importance_dict.items(), key=lambda x: x[1], reverse=True)

#### First we want to remove all values that are less than and equal to zero and save them in a list

removed_pairs_features = []

filtered_pairs_features = [(key, value) for key, value in sorted_features if value > 0]
removed_pairs_features = [(key, value) for key, value in sorted_features if value <= 0]
filtered_pairs_features_numpy = np.array(filtered_pairs_features)
# print("Filtered key-value pairs:", filtered_pairs_features)
# print("Removed key-value pairs:", removed_pairs_features)

In [ ]:
import matplotlib.pyplot as plt

# Extract feature names and importance values for visualization
sorted_feature_names = [x[0] for x in filtered_pairs_features[:20]]
sorted_feature_importance = [x[1] for x in filtered_pairs_features[:20]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_feature_names, sorted_feature_importance)
plt.xlabel('SHAP Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on SHAP Values')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Assuming you have a list of features and their corresponding SHAP values
features = feature_importance_df['Feature'][:20]
shap_values = feature_importance_df['Importance'][:20]

# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(shap_values, features, color='green', s=100, alpha=0.6)
plt.xlabel('SHAP Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
feature_shap_values = [pair[1] for pair in filtered_pairs_features[:30]]
feature_shap_values_names = [pair[0] for pair in filtered_pairs_features[:30]]
feature_names = np.array(feature_shap_values_names)
type(feature_shap_values)

In [ ]:
### Generating the FORCE Plot
shap_values = np.array(feature_shap_values)  # Assuming 100 samples and 5 features
shap_values_ = np.reshape(shap_values, (-1,2))
shap.force_plot(base_values, shap_values, feature_names)

In [ ]:
### Generating the Waterfall Plot

explanation = shap.Explanation(values=shap_values , base_values=base_values, feature_names = feature_names)

shap.waterfall_plot(explanation)

In [ ]:
genotype

In [ ]:
# Sample dataset with a categorical column
data = genotype
df_data = pd.DataFrame(data)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data = pd.DataFrame(data)

# Select features to set one-hot encoding to zero
selected_features = less_important_features

# Set one-hot encoding of selected features to zero
for feature in selected_features:
    for col in df_data.columns:
        if feature in col:
            df_data[col] = 0

# Check the modified dataset
print(df_data)

In [ ]:
#check if less important features are set to zero for one-hot encoding
print(df_data['5014952_chrVIII_185022_G_A'])

In [ ]:
X = df_data
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 0]#index=1 --> 1_CobaltChloride_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
#print(corrupted_X['6386978_chrX_554517_A_T'])

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
def back2labelEncoding(x_list):
    temp = []
    for i in range(x_list.shape[0]):
        data = np.array([t.argmax() for t in x_list[i]])
        temp.append(data)
    return temp

x_train_labelEncoding = np.array(back2labelEncoding(x_train))
print(x_train_labelEncoding.shape)


x_valid_labelEncoding = np.array(back2labelEncoding(x_valid))
print(x_valid_labelEncoding.shape)

x_test_labelEncoding = np.array(back2labelEncoding(x_test))
print(x_test_labelEncoding.shape)
print(y_valid.shape)

In [ ]:
x_train_labelEncoding_ = x_train_labelEncoding.reshape(x_train_labelEncoding.shape[0],x_train_labelEncoding.shape[1],1)
x_valid_labelEncoding_ = x_valid_labelEncoding.reshape(x_valid_labelEncoding.shape[0],x_valid_labelEncoding.shape[1],1)
x_test_labelEncoding_ = x_test_labelEncoding.reshape(x_test_labelEncoding.shape[0],x_test_labelEncoding.shape[1],1)

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,1))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,1))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train_labelEncoding_[:800], 
    [y_train[:]],
    validation_data=(x_valid_labelEncoding_, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
x_valid_labelEncoding_.shape

In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_CobaltChloride_1',0,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
model_mse.save('important features/Models/1_CobaltChloride_1.h5')

In [ ]:
model_mse.summary()

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)



In [ ]:
X_test.shape

In [43]:
#### This is for LIME
import lime
import lime.lime_tabular

print(type(genotype))
genotype_data = genotype.to_numpy()
class_names = ['Positive','Negative']
my_features = genotype.columns.tolist()
#model_mse_ = tf.keras.models.load_model('important features/Models/1_CobaltChloride_1.h5', custom_objects=custom_objects, compile=False)

explainer = lime.lime_tabular.RecurrentTabularExplainer(x_train, mode='regression', training_labels=None, feature_names=my_features, 
                        categorical_features=None, categorical_names=None, kernel_width=None, kernel=None, verbose=False, 
                        class_names=class_names, feature_selection='auto', discretize_continuous=True, discretizer='quartile', 
                         random_state=None)
explainer



<class 'pandas.core.frame.DataFrame'>


In [44]:
model_mse= tf.keras.models.load_model('1_CobaltChloride_1.h5')

i = 0
instance = x_test[i]
exp = explainer.explain_instance(instance, model_mse.predict, num_features=len(my_features))

The initializer HeNormal is unseeded and being called multiple times, which will return identical values each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
2024-07-01 16:04:24.925681: W tensorflow/c/c_api.cc:300] Operation '{name:'dense_1_1/kernel/Assign' id:579 op device:{requested: '', assigned: ''} def:{{{node dense_1_1/kernel/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](dense_1_1/kernel, dense_1_1/kernel/Initializer/stateless_truncated_normal)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.
`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2024

In [45]:
exp.as_list()
lime_importance = dict(exp.as_list())
lime_importance

{'33152_chrI_33152_T_C_t-5765 <= 0.00': -0.0007610346735701451,
 '33152_chrI_33152_T_C_t-23597 <= 0.00': 0.0007557514807891469,
 '0.00 < 33147_chrI_33147_G_T_t-12780 <= 1.00': -0.0007285424862219153,
 '33152_chrI_33152_T_C_t-13032 <= 0.00': -0.0007000744797033448,
 '0.00 < 33147_chrI_33147_G_T_t-13090 <= 1.00': -0.0006957134213626618,
 '33147_chrI_33147_G_T_t-13908 <= 0.00': -0.00069281952645434,
 '33147_chrI_33147_G_T_t-15355 <= 0.00': 0.0006925488591502961,
 '33152_chrI_33152_T_C_t-27214 <= 0.00': -0.0006796023136955452,
 '33147_chrI_33147_G_T_t-22604 <= 0.00': 0.0006665848967681413,
 '0.00 < 33147_chrI_33147_G_T_t-21734 <= 1.00': 0.0006634055773320995,
 '33147_chrI_33147_G_T_t-11323 <= 0.00': 0.0006628225805971143,
 '0.00 < 33147_chrI_33147_G_T_t-11604 <= 1.00': 0.0006584815810542428,
 '0.00 < 33147_chrI_33147_G_T_t-21251 <= 1.00': -0.00065242115973423,
 '0.00 < 33152_chrI_33152_T_C_t-21434 <= 1.00': -0.000650980773555849,
 '0.00 < 33147_chrI_33147_G_T_t-9998 <= 1.00': 0.00064792986

In [48]:
sorted_lime_importance = dict(sorted(lime_importance.items(), key=lambda item: item[1], reverse=True))
sorted_lime_importance

{'33152_chrI_33152_T_C_t-23597 <= 0.00': 0.0007557514807891469,
 '33147_chrI_33147_G_T_t-15355 <= 0.00': 0.0006925488591502961,
 '33147_chrI_33147_G_T_t-22604 <= 0.00': 0.0006665848967681413,
 '0.00 < 33147_chrI_33147_G_T_t-21734 <= 1.00': 0.0006634055773320995,
 '33147_chrI_33147_G_T_t-11323 <= 0.00': 0.0006628225805971143,
 '0.00 < 33147_chrI_33147_G_T_t-11604 <= 1.00': 0.0006584815810542428,
 '0.00 < 33147_chrI_33147_G_T_t-9998 <= 1.00': 0.0006479298623001718,
 '0.00 < 33147_chrI_33147_G_T_t-770 <= 1.00': 0.0006428398918370523,
 '33147_chrI_33147_G_T_t-12579 <= 0.00': 0.0006425523796549798,
 '33147_chrI_33147_G_T_t-23943 <= 0.00': 0.0006371430921352923,
 '0.00 < 33152_chrI_33152_T_C_t-17921 <= 1.00': 0.000628541543982228,
 '33147_chrI_33147_G_T_t-17563 <= 0.00': 0.0006274965210386795,
 '0.00 < 33147_chrI_33147_G_T_t-24327 <= 1.00': 0.0006255030944412278,
 '33152_chrI_33152_T_C_t-21813 <= 0.00': 0.00062235942996029,
 '33147_chrI_33147_G_T_t-8445 <= 0.00': 0.0006217384604151102,
 '0.0

In [50]:
less_important_lime = [k for k, v in lime_importance.items() if v <= 1e-5]
len(less_important_lime)

14930

In [52]:
important_lime = [k for k, v in lime_importance.items() if v >= 1e-5]
len(important_lime)

13290

In [54]:
def find_indexes(main_list, search_list):
    return [i for i, x in enumerate(main_list) if x in search_list]

indexes_lime = find_indexes(lime_importance, important_lime)
print(indexes_lime) 

[1, 6, 8, 9, 10, 11, 14, 17, 18, 20, 24, 25, 27, 28, 29, 30, 31, 33, 36, 38, 41, 45, 46, 49, 51, 54, 56, 57, 59, 63, 66, 68, 71, 72, 76, 78, 81, 88, 91, 93, 96, 97, 98, 99, 104, 105, 107, 110, 113, 115, 117, 120, 122, 125, 127, 131, 132, 133, 136, 139, 140, 141, 144, 146, 147, 148, 152, 156, 157, 158, 159, 160, 162, 163, 167, 168, 169, 171, 172, 173, 174, 175, 178, 179, 182, 184, 187, 189, 190, 192, 196, 198, 199, 203, 208, 211, 212, 214, 216, 217, 218, 220, 226, 228, 230, 233, 234, 235, 237, 239, 240, 245, 250, 253, 258, 259, 267, 270, 272, 273, 275, 277, 278, 279, 287, 288, 293, 294, 298, 299, 302, 304, 305, 308, 310, 312, 314, 317, 320, 322, 323, 326, 330, 332, 334, 335, 337, 341, 342, 344, 348, 350, 352, 353, 354, 356, 359, 363, 365, 366, 368, 369, 370, 372, 376, 377, 380, 383, 384, 385, 390, 395, 399, 404, 407, 408, 410, 418, 419, 420, 422, 427, 429, 432, 433, 439, 440, 443, 446, 450, 452, 456, 457, 459, 460, 463, 464, 471, 473, 479, 480, 484, 488, 489, 490, 491, 493, 494, 497, 49

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]

# Plot the ranked features
plt.figure(figsize=(10, 6))
plt.barh(sorted_lime_names, sorted_lime_importance_)
plt.xlabel('LIME Feature Importance')
plt.ylabel('Feature')
plt.title('Ranked Features based on LIME Values')
plt.show()

In [ ]:
import matplotlib.pyplot as plt

sorted_lime_importance_list = sorted(sorted_lime_importance.items(), key=lambda item: item[1], reverse=True)

num_features_to_plot = min(20, len(sorted_lime_importance_list))

# Extract feature names and importance values for visualization
sorted_lime_names = [x[0] for x in sorted_lime_importance_list[:num_features_to_plot]]
sorted_lime_importance_ = [x[1] for x in sorted_lime_importance_list[:num_features_to_plot]]
# Plotting
plt.figure(figsize=(8, 6))
plt.scatter(sorted_lime_importance_, sorted_lime_names, color='green', s=100, alpha=0.6)
plt.xlabel('LIME Value')
plt.ylabel('Feature')
plt.title('Feature with their corresponding SHAP value')
plt.grid(True, linestyle='--', alpha=0.5)
plt.show()

In [ ]:
# Sample dataset with a categorical column
data_lime = genotype
df_data = pd.DataFrame(data_lime)
# Perform one-hot encoding on a specific categorical column
encoded_df = pd.get_dummies(df_data)

# Display the dataset after one-hot encoding
print("\nDataset after One-Hot Encoding:")
print(encoded_df)
encoded_df.shape

In [ ]:
import pandas as pd

# Sample one-hot encoded dataset
data = encoded_df
df_data_ = pd.DataFrame(data)
type(df_data.columns)
# Select features to set one-hot encoding to zero
selected_features = indexes

# Set one-hot encoding of selected features to zero
for index in selected_features:
    for col in df_data_.columns:
        if index in df_data_.columns:
            df_data_[col] = 0

# Check the modified dataset
print(df_data_)

In [ ]:
X = df_data_
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno
Y = multi_pheno.iloc[:, 0]#index=1 --> 1_CobaltChloride_1
# Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
print(type(corrupted_X_onehot))
#print(corrupted_X['6810072_chrXI_231860_A_G'])
# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

In [ ]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [ ]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [ ]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [ ]:
history_mse,model_mse = runModel_all('1_CobaltChloride_1',0,[1464,608,264,146],[0.00,0.00,0.00,0.20])

In [ ]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


In [ ]:
exp.show_in_notebook(show_table=True)


In [55]:
###### Intersection of LIME and SHAP 

In [76]:
def find_common_indexes(lime_indexes, shap_indexes):
    # Convert lists to sets
    lime_set = set(lime_indexes)
    shap_set = set(shap_indexes)
    
    # Find the intersection
    common_indexes = lime_set.intersection(shap_set)
    
    return common_indexes
    

common_indexes = find_common_indexes(indexes_lime, indexes_shap)
print(type(common_indexes))



<class 'set'>


In [103]:
common_indexes = np.array(list(common_indexes))
(common_indexes)

array([    1,     6,     8, ..., 28190, 28192, 28193])

In [98]:
common_indexes_ = common_indexes.tolist()
len(common_indexes_)

13031

In [99]:
def get_strings_at_indices(strings, indices):

    valid_indices = [index for index in indices if 0 <= index < len(strings)]
    
    # Get strings at valid indices
    result = [strings[index] for index in valid_indices]
    
    return result

matching_features = get_strings_at_indices(feature_importance_df['Feature'].tolist(), common_indexes_)
len(matching_features)

13031

In [104]:
genotypeFile = 'genotype.csv'
genotype = pd.read_csv(genotypeFile, sep = '\t', index_col = 0)
print('genotypeFile shape:', genotype.shape )

phenotypeFile = 'phenotype.csv'
multi_pheno = pd.read_csv(phenotypeFile, sep = ',', index_col = 0)
print('Phenotype_Multi shape:', multi_pheno.shape )


# take a small part to test code
# genotype
X = genotype

mask = ~X.columns.str.contains('|'.join(matching_features))

# Set values to zero for non-matching columns
X.loc[:, mask] = 0

#display(X.head())
# X = genotype.iloc[0:1000:, 0:5000]
# single_pheno

#Chage the index for each model here accoding to the correspinding model
Y = multi_pheno.iloc[:, 0]#index=0 --> 1_CobaltChloride_1
# # Y = multi_pheno.iloc[0:1000, pheno_i]


# # Add noise
# random missing masker
missing_perc = 0.1
nonmissing_ones = np.random.binomial(
    1, 1 - missing_perc, size=X.shape[0] * X.shape[1])
nonmissing_ones = nonmissing_ones.reshape(X.shape[0], X.shape[1])
nonmissing_ones, nonmissing_ones.shape

corrupted_X = X * nonmissing_ones
# corrupted_X.head()

# # Prepare data
# ## One-hot encoding

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
#X_onehot = to_categorical(X)
corrupted_X_onehot = to_categorical(corrupted_X)
# corrupted_X_onehot.shape

# normlization
scaled_Y = (Y - Y.min()) / (Y.max() - Y.min())


def detect_outliers(df):
    outlier_indices = []

    Q1 = np.percentile(df, 25)
    Q3 = np.percentile(df, 75)
    IQR = Q3 - Q1
    outlier_step = 1.5 * IQR

    outlier_indices = df[(df < Q1 - outlier_step) |
                         (df > Q3 + outlier_step)].index

    return outlier_indices


temp_Y = scaled_Y[~scaled_Y.isna()]
outliers_index = detect_outliers(temp_Y)


# set outliers as NAN
scaled_Y_ = scaled_Y.copy()
scaled_Y_[outliers_index] = np.nan


# ## Split train and test
train_X, test_X, corrupted_train_X, corrupted_test_X, train_Y, test_Y = train_test_split(
    X, corrupted_X_onehot, scaled_Y_.iloc[:], test_size=0.1)

# split df to train and valid
train_X, valid_X, corrupted_train_X, corrupted_valid_X, train_Y, valid_Y = train_test_split(
    train_X, corrupted_train_X, train_Y, test_size=0.1)



x_train = to_categorical(pd.concat([train_X,train_Y],axis=1).dropna().iloc[:,0:-1])
y_train = train_Y.dropna().to_numpy()

x_valid = to_categorical(pd.concat([valid_X,valid_Y],axis=1).dropna().iloc[:,0:-1])
y_valid = valid_Y.dropna().to_numpy()

x_test = to_categorical(pd.concat([test_X,test_Y],axis=1).dropna().iloc[:,0:-1])
y_test  = test_Y.dropna().to_numpy()

genotypeFile shape: (4390, 28220)
Phenotype_Multi shape: (4390, 20)


In [105]:
### **Retrain the model on the new dataset**

def model_build(pheno_name,num_hl,hl_list,hl_activation, out_activation, dropout_val,filters_, kernel_size_,stride_poolSize,Conv_Layers_Stride_Size):
    

    '''
    https://towardsdatascience.com/multi-output-model-with-tensorflow-keras-functional-api-875dd89aa7c6

    def DNN_build(num_hl,hl_list,hl_activation, out_activation, dropout_val):
    pheno_name = name of phenotype of interest
    num_hl = number of hidden layers
    hl_list = list of hidden layers
    hl_activation = hidden layer activation function
    out_activation = output layer activation function
    dropout_val = Dropout value
    '''
    assert(num_hl == len(hl_list))
    assert(num_hl == len(dropout_val))
    input_layer = tf.keras.Input(shape=(28220,3))#(3138, 28220, 3)
    
    shared_convL1 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation,input_shape = (28220,3))(input_layer)
    shared_convL1_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL1)
    
    shared_convL2 = tf.keras.layers.Conv1D(filters = filters_,kernel_size = kernel_size_,strides = Conv_Layers_Stride_Size, activation = hl_activation)(shared_convL1_max_pool)
    shared_convL2_max_pool = tf.keras.layers.MaxPool1D(pool_size=stride_poolSize, strides=stride_poolSize)(shared_convL2)
    
    shared_convLayer_Flatten = tf.keras.layers.Flatten()(shared_convL2_max_pool)
    initializer = tf.keras.initializers.HeNormal()
    kernel_regularizer_ = tf.keras.regularizers.L1L2(l1=0.0001, l2=0.0001)
    
    shared_hidden_layer1 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[0], activation  = hl_activation)(shared_convLayer_Flatten)
    shared_hidden_layer1_dp1 = tf.keras.layers.Dropout(dropout_val[0])(shared_hidden_layer1)

    #model 1
    model1_hidden_layer2 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[1], activation  = hl_activation)(shared_hidden_layer1_dp1)
    model1_hidden_layer2_dp2 = tf.keras.layers.Dropout(dropout_val[1])(model1_hidden_layer2)

    model1_hidden_layer3 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[2], activation  = hl_activation)(model1_hidden_layer2_dp2)
    model1_hidden_layer3_dp3 = tf.keras.layers.Dropout(dropout_val[2])(model1_hidden_layer3)

    model1_hidden_layer4 = tf.keras.layers.Dense(kernel_regularizer=kernel_regularizer_,kernel_initializer=initializer,units = hl_list[3], activation  = hl_activation)(model1_hidden_layer3_dp3)
    model1_hidden_layer4_dp4 = tf.keras.layers.Dropout(dropout_val[3])(model1_hidden_layer4)
    model1_hidden_layer4_dp4_fl = tf.keras.layers.Flatten()(model1_hidden_layer4_dp4)
    
    pheno_name_ = tf.keras.layers.Dense(units = 1, name = pheno_name, activation  = out_activation)(model1_hidden_layer4_dp4_fl)#1_CobaltChloride_1
    model = tf.keras.models.Model(inputs = input_layer, outputs = [pheno_name_])

    return model



In [106]:
#ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy','mse'])#change metrics to MSE
def compile_model(pheno_name,model,loss_, learningRate, metrics_): #loss = 'sparse_categorical_crossentropy'
    '''
    def compile_model(DNN, loss_, learningRate, metrics_):
    DNN: the model
    loss_: the loss function
    learningRate: learning rate
    metrics_: metrics of interest ['accuracy', 'mse']
    '''

    # Specify the optimizer, and compile the model with loss functions for both outputs
    model.compile(
       #optimizer = tf.keras.optimizers.Adam(learning_rate=learningRate),#0.00027705 
    optimizer =  tf.keras.optimizers.legacy.SGD(learning_rate=learningRate, momentum=0.01, nesterov=True, name="SGD"),
      loss = {
          pheno_name:loss_
      }, 
      metrics = [metrics_]#{'CobaltChloride_1':[metrics_],'CopperSulfate_1':[metrics_],'Diamide_1':[metrics_]}
      )#change metrics to MSE ##['accuracy','mse']
    return model

def runModel(model, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode,pheno_index):
    '''
    def buildModel(DNN, val_split_size, batch_size_,numEpochs, patience_, monitor_, mode):
    DNN: DNN which the model
    val_split_size: the validation split)
    batch_size_: batch_size
    numEpochs: number of epochs
    patience_: patience of call back
    monitor_: monitor (objective of callback)
    mode: mode (min, max, auto)
    pheno_index: the index of the phenotype in y_train
    '''
    history = model.fit(
    x_train, 
    [y_train[:]],
    validation_data=(x_valid, [y_valid[:]]),
    batch_size = batch_size_, 
    epochs = numEpochs,
    callbacks = [
      tf.keras.callbacks.EarlyStopping(monitor= monitor_,patience=patience_,verbose=1,mode=mode),#monitoring loss mode should be min [---val_acc--]
      #tf.keras.callbacks.ModelCheckpoint(filepath='./TrainedModels/model.{epoch:02d}-{val_loss:.2f}.h5',save_best_only=True),
      #tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    ]
    )

    return history



In [107]:
def runModel_all(pheno_name, pheno_index,hl_list, drop_list):
    model_mse = model_build(pheno_name,4,hl_list,'relu','linear',drop_list,8,7,2,3)#model initiallization
    model_mse = compile_model(pheno_name,model_mse,tf.keras.losses.MeanSquaredError(reduction="auto", name="mse"),
    0.00123695244,'mse')
    history_mse =  runModel(model_mse,0.30,32,400,15, "val_mse", "min",pheno_index)
    return history_mse,model_mse

In [109]:
history_mse,model_mse = runModel_all('1_CobaltChloride_1',0,[1464,608,264,146],[0.00,0.00,0.00,0.20])

Train on 3151 samples, validate on 357 samples


2024-07-02 09:12:06.601198: W tensorflow/c/c_api.cc:300] Operation '{name:'training_2/SGD/conv1d_3_1/bias/momentum/Assign' id:2295 op device:{requested: '', assigned: ''} def:{{{node training_2/SGD/conv1d_3_1/bias/momentum/Assign}} = AssignVariableOp[_has_manual_control_dependencies=true, dtype=DT_FLOAT, validate_shape=false](training_2/SGD/conv1d_3_1/bias/momentum, training_2/SGD/conv1d_3_1/bias/momentum/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Epoch 1/400
3104/3151 [============================>.] - ETA: 0s - loss: 17.7185 - mse: 0.0333

2024-07-02 09:12:08.669318: W tensorflow/c/c_api.cc:300] Operation '{name:'loss_2/AddN_1' id:2075 op device:{requested: '', assigned: ''} def:{{{node loss_2/AddN_1}} = AddN[N=2, T=DT_FLOAT, _has_manual_control_dependencies=true](loss_2/mul, loss_2/AddN)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


3151/3151 [==============================] - 2s 563us/sample - loss: 17.7184 - mse: 0.0333 - val_loss: 17.6903 - val_mse: 0.0121
Epoch 2/400
3151/3151 [==============================] - 2s 477us/sample - loss: 17.6931 - mse: 0.0216 - val_loss: 17.6754 - val_mse: 0.0109
Epoch 3/400
3151/3151 [==============================] - 2s 478us/sample - loss: 17.6764 - mse: 0.0186 - val_loss: 17.6611 - val_mse: 0.0102
Epoch 4/400
3151/3151 [==============================] - 1s 471us/sample - loss: 17.6604 - mse: 0.0161 - val_loss: 17.6472 - val_mse: 0.0098
Epoch 5/400
3151/3151 [==============================] - 2s 478us/sample - loss: 17.6459 - mse: 0.0152 - val_loss: 17.6333 - val_mse: 0.0094
Epoch 6/400
3151/3151 [==============================] - 2s 477us/sample - loss: 17.6313 - mse: 0.0141 - val_loss: 17.6194 - val_mse: 0.0091
Epoch 7/400
3151/3151 [==============================] - 1s 475us/sample - loss: 17.6173 - mse: 0.0136 - val_loss: 17.6057 - val_mse: 0.0089
Epoch 8/400
3151/3151 [==

In [110]:
from sklearn.metrics import mean_squared_error

# Make predictions on the validation set
y_pred = model_mse.predict(x_valid)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_valid, y_pred)
print("Mean Squared Error on the validation set:", mse)

# Make predictions on the tesing set
y_pred = model_mse.predict(x_test)

# Calculate the Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print("Mean Squared Error on the testing set:", mse)


`Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
2024-07-02 09:26:20.556257: W tensorflow/c/c_api.cc:300] Operation '{name:'1_CobaltChloride_1_3/BiasAdd' id:1926 op device:{requested: '', assigned: ''} def:{{{node 1_CobaltChloride_1_3/BiasAdd}} = BiasAdd[T=DT_FLOAT, _has_manual_control_dependencies=true, data_format="NHWC"](1_CobaltChloride_1_3/MatMul, 1_CobaltChloride_1_3/BiasAdd/ReadVariableOp)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Mean Squared Error on the validation set: 0.006473201335103157
Mean Squared Error on the testing set: 0.0075092416484671


In [ ]:
import matplotlib.pyplot as plt

# Sample data
x1 = ['1_Coba1tChloride_1','1_CopperSulfate_1','1_Diamide_1','1_E6-Berbamine_1','1_Ethanol_1','1_Formamide_1','1_Hydroxyurca_1',
      '1_Indo1aceticAcid_1','1_Lactate_1','1_Lactose_1','1_MagncsiumChloride_1','1_ManganeseSulfate_1','1_Menadione_1','1_Neomycin_1',
      '1_Raffinosc_1','1_Trehalose_1','1_Xylose_1','1_YNB_1','1_YPD_1','1_Zeocin_1']
y1 = [0.0071, 0.0036, 0.0180, 0.0144, 0.0061, 0.0032, 0.0037, 0.0066, 0.0036, 0.0031, 0.0050, 0.0107, 0.0045,
      0.0052, 0.0066, 0.0053, 0.0061, 0.0077, 0.0054, 0.0201]

x2 = ['1_Coba1tChloride_1','1_CopperSulfate_1','1_Diamide_1','1_E6-Berbamine_1','1_Ethanol_1','1_Formamide_1','1_Hydroxyurca_1',
      '1_Indo1aceticAcid_1','1_Lactate_1','1_Lactose_1','1_MagncsiumChloride_1','1_ManganeseSulfate_1','1_Menadione_1','1_Neomycin_1',
      '1_Raffinosc_1','1_Trehalose_1','1_Xylose_1','1_YNB_1','1_YPD_1','1_Zeocin_1']
y2 = [0.0070, 0.0041, 0.0181, 0.0116, 0.0094, 0.0039, 0.00398, 0.0066, 0.0060, 0.0044, 0.0061, 0.0102, 0.0043, 
      0.0064, 0.0071, 0.0043, 0.0060, 0.0079, 0.0049, 0.0122]

x3 = ['1_Coba1tChloride_1','1_CopperSulfate_1','1_Diamide_1','1_E6-Berbamine_1','1_Ethanol_1','1_Formamide_1','1_Hydroxyurca_1',
      '1_Indo1aceticAcid_1','1_Lactate_1','1_Lactose_1','1_MagncsiumChloride_1','1_ManganeseSulfate_1','1_Menadione_1','1_Neomycin_1',
      '1_Raffinosc_1','1_Trehalose_1','1_Xylose_1','1_YNB_1','1_YPD_1','1_Zeocin_1']
y3 =  [0.0075, 0.0072, 0.0194, 0.0077, 0.0062, 0.0036, 0.0037, 0.0049, 0.0044, 0.0052, 0.0062, 0.0104, 0.0088, 0.0059, 0.0075, 0.0087, 
      0.0065, 0.008, 0.0068, 0.0177]

x4 =  ['1_Coba1tChloride_1','1_CopperSulfate_1','1_Diamide_1','1_E6-Berbamine_1','1_Ethanol_1','1_Formamide_1','1_Hydroxyurca_1',
      '1_Indo1aceticAcid_1','1_Lactate_1','1_Lactose_1','1_MagncsiumChloride_1','1_ManganeseSulfate_1','1_Menadione_1','1_Neomycin_1',
      '1_Raffinosc_1','1_Trehalose_1','1_Xylose_1','1_YNB_1','1_YPD_1','1_Zeocin_1']

y4 =  [0.0061, 0.0045, 0.0196, 0.01264, 0.00814, 0.004283, 0.00387, 0.00636, 0.00494, 0.00635, 0.006223, 0.01093, 0.00899, 0.00594,
       0.00673, 0.007708, 0.00673, 0.00696, 0.0074, 0.0162]

# Create the plot
plt.figure(figsize=(15, 8))  # Set the figure size

# Plot the first dataset as a line graph
plt.plot(x1, y1, linestyle='-', color='b', label='QPPCNN_test on Important Features')

# Plot the second dataset as a line graph
plt.plot(x2, y2, linestyle='--', color='b', label='QPPCNN_validation on Important Features')

# Plot the third dataset as a line graph
plt.plot(x3, y3, linestyle='-', color='r', label='QPPCNN_test on All Features')

# Plot the fourth dataset as a line graph
plt.plot(x4, y4, linestyle='--', color='r', label='QPPCNN_validation on All Features')


# Add labels and title
plt.xlabel('Phenotype')
plt.ylabel('Mean Square Error')

# Rotate x-axis labels for better readability
plt.xticks(rotation=90)

# Add legend
plt.legend()

# Show the plot
plt.grid(False)
plt.tight_layout()  # Adjust layout to prevent overlap of labels
plt.show()